In [1]:
import findspark
findspark.find()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("BigdataPlatform").config("spark.sql.repl.eagerEval.enabled", True).getOrCreate()

In [2]:
# 데이터 확인
df_19 = spark.read.csv("./dataset/국회입법예고_복지법_19.csv", header=True, inferSchema=True)
df_20 = spark.read.csv("./dataset/국회입법예고_복지법_20.csv", header=True, inferSchema=True)
df_21 = spark.read.csv("./dataset/국회입법예고_복지법_21.csv", header=True, inferSchema=True)

df_19.show()
df_20.show()
df_21.show()

+--------------------------+-----------------------------------+--------------------------------+
|                  law_name|                            content|                        opinions|
+--------------------------+-----------------------------------+--------------------------------+
|[1918675] 아동복지법 일...|  최근 우리사회에 다양한 형태의 ...|  ['반대합니다.', '반대합니다...|
|[1918660] 아동복지법 일...|  ■ 제안이유 보건복지부가 발표한...|['피해 아동뿐 아니라 피해가족...|
|[1918210] 장애인복지법 ...|  제안이유 및 주요내용 현행법은 ...|                       의견 없음|
|[1918207] 노인복지법 일...|      ■ 제안이유 및 주요내용 201...|                       의견 없음|
|[1918172] 장애인복지법 ...|  ■ 제안이유 청각장애인과 언어장...|                       의견 없음|
|[1918171] 노인복지법 일...|    제안이유 65세 이상 노인들은 ...|                       의견 없음|
|[1918169] 노인복지법 일...|      ■ 제안이유 및 주요내용 201...|                       의견 없음|
|[1918022] 노인복지법 일...|노인복지주택은 노인의 주거안정을...|                       의견 없음|
|[1918020] 장애인복지법 ...|   ■ 제안이유 장애인과 함께 생활...|                       의견 없음|
|[1917828] 장애인복지법 ...|  제안이유 장애인이 장애

In [3]:
# '의견 없음' 행 추출 및 개수 확인
filtered_df19 = df_19.filter(df_19["opinions"] == "의견 없음")
filtered_df20 = df_20.filter(df_20["opinions"] == "의견 없음")
filtered_df21 = df_21.filter(df_21["opinions"] == "의견 없음")

print(f"제 19대 법률안 중 의견 없음 개수: {filtered_df19.count()}")
print(f"제 20대 법률안 중 의견 없음 개수: {filtered_df20.count()}")
print(f"제 21대 법률안 중 의견 없음 개수: {filtered_df21.count()}")

제 19대 법률안 중 의견 없음 개수: 176
제 20대 법률안 중 의견 없음 개수: 0
제 21대 법률안 중 의견 없음 개수: 0


In [4]:
from pyspark.sql.functions import col, count, regexp_extract

pattern = r'\[(.*?)\]'  
extracted = regexp_extract(filtered_df19["law_name"], pattern, 1) 
keywords = ['노인', '아동', '장애인'] 

for keyword in keywords:
    filtered_df = df_19.filter((col("law_name").contains(keyword)) & (df_19["opinions"] == "의견 없음"))
    count_by_keyword = filtered_df.count()
    print(f"'{keyword}' 별 의견 없음 행의 개수: {count_by_keyword}")

'노인' 별 의견 없음 행의 개수: 88
'아동' 별 의견 없음 행의 개수: 0
'장애인' 별 의견 없음 행의 개수: 88


In [8]:
from pyspark.sql.functions import col, size, desc, split

for df_name, df in [("19", df_19), ("20", df_20), ("21", df_21)]:
    split_opinions = split(df["opinions"], ",")
    opinions_count = size(split_opinions)
    max_opinions_count = df.withColumn("opinions_count", opinions_count).agg({"opinions_count": "max"}).collect()[0][0]
    max_opinions_law = df.withColumn("opinions_count", opinions_count).filter(col("opinions_count") == max_opinions_count).first()
    law_name = max_opinions_law["law_name"]
    print(f"{df_name} 대 중 가장 많은 의견을 가진 법률안: {law_name}, 의견 수: {max_opinions_count}")

19 대 중 가장 많은 의견을 가진 법률안: [1918675] 아동복지법 일부개정법률안(황인자의원 등 10인), 의견 수: 210
20 대 중 가장 많은 의견을 가진 법률안: [2024360] 아동복지법 일부개정법률안(정은혜의원 등 10인), 의견 수: 270
21 대 중 가장 많은 의견을 가진 법률안: [2125786] 노인복지법 일부개정법률안(이수진의원 등 10인), 의견 수: 279


In [17]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, size, udf
from pyspark.sql.types import IntegerType 

def count_opinions(opinions_list):
    counter = 0
    for opinion in opinions_list:
        if '반대' in opinion:
            counter += 1
    return counter

count_opinions_udf = F.udf(count_opinions, IntegerType())

# 19대, 20대, 21대 각각에 대해 '반대' 의견이 가장 많은 법률안 찾기
for df_name, df in [("19", df_19), ("20", df_20), ("21", df_21)]:
    df_with_oppose_count = df.withColumn("opinions_list", split(col("opinions"), ", ")) \
        .withColumn("oppose_count", count_opinions_udf(col("opinions_list"))) \
        .withColumn("total_opinions", size(col("opinions_list")))

    # '반대' 의견이 가장 많은 법률안 찾기
    max_oppose_law = df_with_oppose_count.orderBy(col("oppose_count").desc()).first()
    law_name = max_oppose_law["law_name"]
    total_opinions = max_oppose_law["total_opinions"]
    oppose_count = max_oppose_law["oppose_count"]
    oppose_ratio = (oppose_count / total_opinions) * 100

    print(f"{df_name}대 가장 많은 반대 의견을 받은 법률안: {law_name}, 총 의견 수: {total_opinions}, 반대 의견 수: {oppose_count}, 반대 비율: {oppose_ratio:.2f}%")

19대 가장 많은 반대 의견을 받은 법률안: [1918675] 아동복지법 일부개정법률안(황인자의원 등 10인), 총 의견 수: 210, 반대 의견 수: 203, 반대 비율: 96.67%
20대 가장 많은 반대 의견을 받은 법률안: [2024360] 아동복지법 일부개정법률안(정은혜의원 등 10인), 총 의견 수: 270, 반대 의견 수: 258, 반대 비율: 95.56%
21대 가장 많은 반대 의견을 받은 법률안: [2125786] 노인복지법 일부개정법률안(이수진의원 등 10인), 총 의견 수: 279, 반대 의견 수: 270, 반대 비율: 96.77%
